In [1]:
import time
import snscrape.modules.twitter as sntwitter
import pandas as pd
import os

In [2]:
start = time.time()

# On crée la liste qui contiendra les informations des tweets
tweets_list = []

# On choisit les périodes que l'on veut scraper en fonction des dates de sortie des trailers et des films
periods_dune = [pd.date_range(start='2020-09-09', end='2020-09-23').strftime("%Y-%m-%d").tolist(),
                pd.date_range(start='2021-07-22', end='2021-08-05').strftime("%Y-%m-%d").tolist(),
                pd.date_range(start='2021-10-08', end='2021-11-13').strftime("%Y-%m-%d").tolist()]

periods_space= [pd.date_range(start='2021-04-03', end='2021-04-17').strftime("%Y-%m-%d").tolist(),
                pd.date_range(start='2021-06-09', end='2021-06-23').strftime("%Y-%m-%d").tolist(),
                pd.date_range(start='2021-07-03', end='2021-08-08').strftime("%Y-%m-%d").tolist()]

# On utilise TwitterSearchScraper pour obtenir tous les tweets contenant un string choisi durant une période choisie
for film , periods in [('dune',periods_dune),('space jam',periods_space)]:
    for time_range in periods :
        for k in range(len(time_range)-1):
            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f"{film} movie since:{time_range[k]} until:{time_range[k+1]} lang:en").get_items()):
                if i>=500: # On scrape 500 tweets par jour
                    break
                tweets_list.append([film, time_range, tweet.content, tweet.date, tweet.id, tweet.lang]) # On prend les informations intéressantes
    
# On crée un dataframe à partir de ce qu'on a scrapé
tweets_df = pd.DataFrame(tweets_list, columns=['Film', 'Time Range', 'Text', 'Datetime', 'Tweet Id', 'Language'])
end=time.time()
end-start

1343.0095012187958

In [3]:
pd.set_option('display.max_colwidth', 550)

In [4]:
tweets_df

,Film,Time Range,Text,Datetime,Tweet Id,Language
0,dune,"[2020-09-09, 2020-09-10, 2020-09-11, 2020-09-12, 2020-09-13, 2020-09-14, 2020-09-15, 2020-09-16, 2020-09-17, 2020-09-18, 2020-09-19, 2020-09-20, 2020-09-21, 2020-09-22, 2020-09-23]","I wanna see Dune but the lack of Muslim and MENA actors when the novel supposedly references the middle-east and Islam is.... disappointing. Would have been a real good bit of representation for once, especially in a big Hollywood movie",2020-09-09 23:59:53+00:00,1303845614120304641,en
1,dune,"[2020-09-09, 2020-09-10, 2020-09-11, 2020-09-12, 2020-09-13, 2020-09-14, 2020-09-15, 2020-09-16, 2020-09-17, 2020-09-18, 2020-09-19, 2020-09-20, 2020-09-21, 2020-09-22, 2020-09-23]",I might be the only person that knows fuckall about Dune but damn that trailer looks great and I love Denis Villenueve. That dude hasn't made a bad movie that I've seen and I've seen almost all of them.,2020-09-09 23:59:48+00:00,1303845592548941824,en
2,dune,"[2020-09-09, 2020-09-10, 2020-09-11, 2020-09-12, 2020-09-13, 2020-09-14, 2020-09-15, 2020-09-16, 2020-09-17, 2020-09-18, 2020-09-19, 2020-09-20, 2020-09-21, 2020-09-22, 2020-09-23]","When people try to claim Dune is some higher level, intelligent adult sci-fi and I wonder if I watched the same movie? #Dune https://t.co/rHiAbRLQL0",2020-09-09 23:59:43+00:00,1303845572814790656,en
3,dune,"[2020-09-09, 2020-09-10, 2020-09-11, 2020-09-12, 2020-09-13, 2020-09-14, 2020-09-15, 2020-09-16, 2020-09-17, 2020-09-18, 2020-09-19, 2020-09-20, 2020-09-21, 2020-09-22, 2020-09-23]",@nosleeptilltaco I never read Dune but Im v glad yall that did are getting a good movie,2020-09-09 23:59:40+00:00,1303845558373756929,en
4,dune,"[2020-09-09, 2020-09-10, 2020-09-11, 2020-09-12, 2020-09-13, 2020-09-14, 2020-09-15, 2020-09-16, 2020-09-17, 2020-09-18, 2020-09-19, 2020-09-20, 2020-09-21, 2020-09-22, 2020-09-23]","@dunemovie The mix of styles is really strange , the music is not matching the clip or the ambiance , almost disgusting me , also the casting isn't that great imo for a movie with such ambition .. I have that feeling telling me it's gonna be an okay movie and not more",2020-09-09 23:59:39+00:00,1303845555169374214,en
...,...,...,...,...,...,...
44856,space jam,"[2021-07-03, 2021-07-04, 2021-07-05, 2021-07-06, 2021-07-07, 2021-07-08, 2021-07-09, 2021-07-10, 2021-07-11, 2021-07-12, 2021-07-13, 2021-07-14, 2021-07-15, 2021-07-16, 2021-07-17, 2021-07-18, 2021-07-19, 2021-07-20, 2021-07-21, 2021-07-22, 2021-07-23, 2021-07-24, 2021-07-25, 2021-07-26, 2021-07-27, 2021-07-28, 2021-07-29, 2021-07-30, 2021-07-31, 2021-08-01, 2021-08-02, 2021-08-03, 2021-08-04, 2021-08-05, 2021-08-06, 2021-08-07, 2021-08-08]",we went to see space jam.... lebron james is not the greatest actor in the world but it was still a cute movie,2021-08-07 00:29:43+00:00,1423803505463156738,en
44857,space jam,"[2021-07-03, 2021-07-04, 2021-07-05, 2021-07-06, 2021-07-07, 2021-07-08, 2021-07-09, 2021-07-10, 2021-07-11, 2021-07-12, 2021-07-13, 2021-07-14, 2021-07-15, 2021-07-16, 2021-07-17, 2021-07-18, 2021-07-19, 2021-07-20, 2021-07-21, 2021-07-22, 2021-07-23, 2021-07-24, 2021-07-25, 2021-07-26, 2021-07-27, 2021-07-28, 2021-07-29, 2021-07-30, 2021-07-31, 2021-08-01, 2021-08-02, 2021-08-03, 2021-08-04, 2021-08-05, 2021-08-06, 2021-08-07, 2021-08-08]",My favorite Space Jam 2 critique wasn't even a direct critique of the movie; it was just some guy complaining about Rick and Morty not swearing in the movie for children.,2021-08-07 00:20:27+00:00,1423801175372087298,en
44858,space jam,"[2021-07-03, 2021-07-04, 2021-07-05, 2021-07-06, 2021-07-07, 2021-07-08, 2021-07-09, 2021-07-10, 2021-07-11, 2021-07-12, 2021-07-13, 2021-07-14, 2021-07-15, 2021-07-16, 2021-07-17, 2021-07-18, 2021-07-19, 2021-07-20, 2021-07-21, 2021-07-22, 2021-07-23, 2021-07-24, 2021-07-25, 2021-07-26, 2021-07-27, 2021-07-28, 2021-07-29, 2021-07-30, 2021-07-31, 2021-08-01, 2021-08-02, 2021-08-03, 2021-08-04, 2021-08-05, 2021-08-06, 2021-08-07, 2021-08-08]","@James

In [6]:
file = os.path.join("data", "web", "web.csv")
if not os.path.exists(os.path.dirname(file)):
    os.makedirs(os.path.dirname(file))
tweets_df.to_pickle(file)